# NFL Big Data Bowl 2024

## Importing dataview libraries

In [83]:
import pandas as pd
import numpy as np

In [3]:
DATA_ROOT = '../data/'

In [5]:
games = pd.read_csv(DATA_ROOT + 'games.csv')
players = pd.read_csv(DATA_ROOT + 'players.csv')
plays = pd.read_csv(DATA_ROOT + 'plays.csv')
tackles = pd.read_csv(DATA_ROOT + 'tackles.csv')

In [6]:
# Modify the plays dataframe to add a column for the play direction (pass short middle, run left end, etc.) and a column for if the play was in shotgun formation.
import re

def playDescriptionToDirection(play_description):
    play = re.search('(pass (?:short|deep) (?:left|middle|right))|((?:left|right) (?:guard|tackle|end))|(up the middle)', play_description)
    return play.group(0) if play else 'FUMBLED BALL'

def playDescriptionToShotgun(play_description):
    return re.search('[(]Shotgun[)]', play_description) != None

def playDescriptionToStartingYard(play_description, offense):
    if re.search('TOUCHDOWN', play_description):
        return 100 - int(re.search('(?<=for )\d+', play_description).group(0))
    if re.search('SAFETY', play_description):
        return 100 + int(re.search('(?<=for -)\d+', play_description).group(0))
    match = re.search('(?:(?:to|at)\s)(?:(?:(\w+)(?:\s)(\d+))|(50))(?: for )(?:(?:([-]?\d+)(?: yard))|(no gain))', play_description)
    if match == None:
        print('WARN: Couldn\'t parse:', play_description)
        return -1
    end_yard = -1
    if match.group(3) == '50':
        end_yard = 50
    else:
        end_yard = int(match.group(2))
        end_yard = 100 - end_yard if match.group(1) != offense else end_yard
    if match.group(5) == 'no gain':
        return end_yard
    return end_yard - int(match.group(4))
    

In [42]:
print('Direction')
plays['playDirection'] = plays['playDescription'].map(playDescriptionToDirection)

plays["offenseFormation"] = plays["offenseFormation"].astype('category')
plays["offenseFormationCategory"] = plays["offenseFormation"].cat.codes

Direction


In [38]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [100]:
X = plays[['down', 'yardsToGo', 'offenseFormationCategory', 'defendersInTheBox', 'absoluteYardlineNumber']]
Y = plays['playDirection']
X = pd.get_dummies(X, columns=['offenseFormationCategory'])
Y = pd.get_dummies(Y, columns=['playDirection'])
X = X.values.astype(float)
Y = Y.values
print(X)

[[ 1. 10.  4.  7. 69.]
 [ 1. 10.  4.  7. 76.]
 [ 2.  5.  1.  6. 40.]
 ...
 [ 1. 10.  4.  6. 69.]
 [ 1. 10.  4.  6. 44.]
 [ 3.  1.  1.  8. 27.]]


In [92]:
# define baseline model
def baseline_model():
 # create model
 model = Sequential()
 model.add(Dense(8, input_dim=4, activation='relu'))
 model.add(Dense(3, activation='softmax'))
 # Compile model
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 return model

In [93]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=1)

In [94]:
kfold = KFold(n_splits=10, shuffle=True)

In [95]:
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/masonwilde/Dev/bonfire-big-data-bowl-2024/venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/masonwilde/Dev/bonfire-big-data-bowl-2024/venv/lib/python3.11/site-packages/scikeras/wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "/Users/masonwilde/Dev/bonfire-big-data-bowl-2024/venv/lib/python3.11/site-packages/scikeras/wrappers.py", line 760, in fit
    self._fit(
  File "/Users/masonwilde/Dev/bonfire-big-data-bowl-2024/venv/lib/python3.11/site-packages/sci